In [1]:
import sys
import numpy as np
import pandas as pd
import PIL
print(sys.version)
print(np.__version__)
print(pd.__version__)
print(PIL.__version__)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
1.25.2
2.1.0
10.0.0


In [6]:
from Map import Map_Obj

In [18]:
themap = Map_Obj(task=4)

In [10]:
themap.show_map()

In [11]:
themap.int_map.shape

(47, 39)

In [24]:
def h(a: list, b: list)-> float:
    """
    Heuristic function h

    parameters
    ---------
    a: list 
        current position
    b: list
        goal position
    returns
    -------
    float
        heuristic value
        manhattan distance from current position to goal position
    """
    x1, y1 = a[0], a[1]
    x2, y2 = b[0], b[1]
    distance=abs(x1 - x2) + abs(y1 - y2)
    #print(distance)
    return distance

In [25]:
print(h(themap.get_start_pos(), themap.get_goal_pos()))

22
22


In [23]:
print(themap.get_start_pos())
print(themap.get_goal_pos())

[28, 32]
[6, 32]
